# Imports

In [1]:
import random
import pandas as pd
import numpy as np
from copy import deepcopy
from pprint import pprint

# Load Data

In [2]:
df = pd.read_csv("data/players.csv")
df = df.drop(columns=['Unnamed: 0'])

# Problem Configuration

In [3]:
TEAM_SIZE = 7
NUM_TEAMS = 5
BUDGET_LIMIT = 750
TEAM_STRUCTURE = {"GK": 1, "DEF": 2, "MID": 2, "FWD": 2}
POPULATION_SIZE = 10

In [4]:
# Group players by position
players_by_position = {
    "GK": df[df['Position'] == "GK"].to_dict('records'),
    "DEF": df[df['Position'] == "DEF"].to_dict('records'),
    "MID": df[df['Position'] == "MID"].to_dict('records'),
    "FWD": df[df['Position'] == "FWD"].to_dict('records')
}

In [5]:
# print players by each position
for position, players in players_by_position.items():
    print(f"\n--- {position} ({len(players)} players) ---")
    for player in players:
        print(f"Name: {player['Name']}, Skill: {player['Skill']}, Salary: €{player['Salary (€M)']}M")



--- GK (5 players) ---
Name: Alex Carter, Skill: 85, Salary: €90M
Name: Jordan Smith, Skill: 88, Salary: €100M
Name: Ryan Mitchell, Skill: 83, Salary: €85M
Name: Chris Thompson, Skill: 80, Salary: €80M
Name: Blake Henderson, Skill: 87, Salary: €95M

--- DEF (10 players) ---
Name: Daniel Foster, Skill: 90, Salary: €110M
Name: Lucas Bennett, Skill: 85, Salary: €90M
Name: Owen Parker, Skill: 88, Salary: €100M
Name: Ethan Howard, Skill: 80, Salary: €70M
Name: Mason Reed, Skill: 82, Salary: €75M
Name: Logan Brooks, Skill: 86, Salary: €95M
Name: Caleb Fisher, Skill: 84, Salary: €85M
Name: Maxwell Flores, Skill: 81, Salary: €72M
Name: Jaxon Griffin, Skill: 79, Salary: €65M
Name: Brayden Hughes, Skill: 87, Salary: €100M

--- MID (10 players) ---
Name: Nathan Wright, Skill: 92, Salary: €120M
Name: Connor Hayes, Skill: 89, Salary: €105M
Name: Dylan Morgan, Skill: 91, Salary: €115M
Name: Hunter Cooper, Skill: 83, Salary: €85M
Name: Austin Torres, Skill: 82, Salary: €80M
Name: Gavin Richardson, S

In [6]:
# create a valid team
def create_valid_team(available_players):
    team = []
    used_ids = set()
    for pos, count in TEAM_STRUCTURE.items():
        candidates = [p for p in available_players[pos] if id(p) not in used_ids]
        if len(candidates) < count:
            return None, set()  # não há jogadores suficientes
        selected = random.sample(candidates, count)
        for player in selected:
            used_ids.add(id(player))
            team.append(player)
    return team, used_ids

In [7]:
# print a valid team just for testing
team, used_ids = create_valid_team(players_by_position)
if team:
    for player in team:
        print(f"{player['Position']}: {player['Name']} | Skill: {player['Skill']} | Salary: €{player['Salary (€M)']}M")
else:
    print("Failed to create a valid team.")

GK: Alex Carter | Skill: 85 | Salary: €90M
DEF: Brayden Hughes | Skill: 87 | Salary: €100M
DEF: Ethan Howard | Skill: 80 | Salary: €70M
MID: Dominic Bell | Skill: 86 | Salary: €95M
MID: Austin Torres | Skill: 82 | Salary: €80M
FWD: Xavier Bryant | Skill: 90 | Salary: €120M
FWD: Tyler Jenkins | Skill: 80 | Salary: €70M


In [8]:
# create a league = one individual = one possible solution
def create_league():
    all_players = deepcopy(players_by_position)
    league = []
    used_ids_total = set() # tracks all players used so far across all teams to avoid duplication
    for _ in range(NUM_TEAMS):
        team, used_ids = create_valid_team(all_players)
        if team is None or sum(p['Salary (€M)'] for p in team) > BUDGET_LIMIT:
            return None  # invalid team
        league.append(team)
        used_ids_total.update(used_ids)
        for pos in players_by_position:
            all_players[pos] = [p for p in all_players[pos] if id(p) not in used_ids_total] # remove used players
    return league

In [9]:
league = create_league()

if league is None:
    print("❌ Failed to generate a valid league.")
else:
    for team_index, entry in enumerate(league, start=1):
        print(f"\n🏆 Team {team_index}")
        for player in entry["Players"]:
            print(f"  {player['Position']} - {player['Name']} | Skill: {player['Skill']} | Salary: €{player['Salary (€M)']}M")



🏆 Team 1


TypeError: list indices must be integers or slices, not str

In [ ]:
# ======================
# VALIDAR EQUIPA
# ======================
def is_valid_team(team):
    if len(team) != TEAM_SIZE:
        return False
    pos_counts = {"GK": 0, "DEF": 0, "MID": 0, "FWD": 0}
    total_salary = 0
    ids = set()
    for p in team:
        pos_counts[p['Position']] += 1
        total_salary += p['Salary (€M)']
        ids.add(p['Name'])
    return pos_counts == TEAM_STRUCTURE and total_salary <= BUDGET_LIMIT and len(ids) == TEAM_SIZE

In [ ]:
# ======================
# FITNESS FUNCTION
# ======================
def fitness(individual):
    if individual is None:
        return float('inf')
    means = []
    used_names = set()
    for team in individual:
        if not is_valid_team(team):
            return float('inf')
        for p in team:
            if p['Name'] in used_names:
                return float('inf')  # jogador repetido
            used_names.add(p['Name'])
        avg_skill = np.mean([p['Skill'] for p in team])
        means.append(avg_skill)
    return np.std(means)  # quanto menor, melhor

In [ ]:
# ======================
# GERAR POPULAÇÃO INICIAL
# ======================
def generate_initial_population(size=POPULATION_SIZE):
    population = []
    while len(population) < size:
        indiv = create_individual()
        if indiv is not None:
            population.append(indiv)
    return population

